# Install all the dependencies

In [1]:
! pip install git+https://github.com/openai/whisper.git -q
! pip install -q git+https://github.com/pyannote/pyannote-audio
! pip install --quiet openai
! pip install gradio -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [18]:
! pip install textstat -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.6 MB/s eta 0:00:00


# Download and load the model

In [3]:
from whisper import _download, _MODELS

_download(_MODELS["medium"], "/models/", False)

100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 113MiB/s]


'/models/medium.pt'

In [5]:
import whisper
model = whisper.load_model('/models/medium.pt')


# Convert the mp3 audio to .wav

In [4]:
import wave
import subprocess

audio = "/content/customer_support.mp3"
if audio[-3:] != 'wav':
  audio_file_name = audio.split("/")[-1]
  audio_file_name = audio_file_name.split(".")[0] + ".wav"
  subprocess.call(['ffmpeg', '-i', audio, audio_file_name, '-y'])
  path = audio_file_name

In [6]:
result = model.transcribe(path)
segments = result["segments"]

print(segments)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.48, 'text': " Thank you for calling Martha's Flowers Towne SST. Hello I'd like to order flowers", 'tokens': [50364, 1044, 291, 337, 5141, 27787, 311, 48194, 15954, 68, 318, 6840, 13, 2425, 286, 1116, 411, 281, 1668, 8085, 50638], 'temperature': 0.0, 'avg_logprob': -0.26045784723191034, 'compression_ratio': 1.5639810426540284, 'no_speech_prob': 0.6478767991065979}, {'id': 1, 'seek': 0, 'start': 5.48, 'end': 9.040000000000001, 'text': " and I think you have what I'm looking for. I'd be happy to take care of your", 'tokens': [50638, 293, 286, 519, 291, 362, 437, 286, 478, 1237, 337, 13, 286, 1116, 312, 2055, 281, 747, 1127, 295, 428, 50816], 'temperature': 0.0, 'avg_logprob': -0.26045784723191034, 'compression_ratio': 1.5639810426540284, 'no_speech_prob': 0.6478767991065979}, {'id': 2, 'seek': 0, 'start': 9.040000000000001, 'end': 14.48, 'text': ' order may have your name please. Randall Thomas. Randall Thomas can you spell that', 'tokens': [50

In [7]:
import contextlib

with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

# Load the embedding model to perform diarization

In [8]:
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
import torch

embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

(…)a-voxceleb/resolve/main/hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

(…)eleb/resolve/main/mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

(…)-voxceleb/resolve/main/label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [9]:
from pyannote.audio import Audio
from pyannote.core import Segment
import numpy as np

audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  waveform = waveform.mean(dim=0, keepdim=True)
  return embedding_model(waveform.unsqueeze(0))


embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [10]:
from sklearn.cluster import AgglomerativeClustering

num_speakers = 2  # Set the number of speakers manually

clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [11]:
import datetime

def time(secs):
  return datetime.timedelta(seconds=round(secs))

conversation = ""

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    conversation += "\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n'
  conversation += segment["text"][1:] + ' '


print(conversation)



SPEAKER 1 0:00:00
Thank you for calling Martha's Flowers Towne SST. Hello I'd like to order flowers 
SPEAKER 2 0:00:05
and I think you have what I'm looking for. I'd be happy to take care of your 
SPEAKER 1 0:00:09
order may have your name please. Randall Thomas. Randall Thomas can you spell that 
SPEAKER 2 0:00:14
for me? Randall R-A-N-B-A-L-L Thomas T-H-O-M-A-S. Thank you for that information Randall may have your home or office number area code first. Area code 409 then 866-5088. 
SPEAKER 1 0:00:34
That's 409-866-5088. Do you have a fax number or email address? Email is Randall.Thomas.Gmail.com. 
SPEAKER 2 0:00:43
Randall.Thomas.Gmail.com may have your shipping address. 6800. Gladys Avenue, Beaumont Texas, zip code is 77706. Gladys Avenue, Beaumont Texas, zip code 77706. 
SPEAKER 1 0:01:06
Thank you for the information. What products are you interested in purchasing? Red roses, probably a dozen. One dozen of red roses, do you want long stems? Sure. Alright Randall let me process yo

# Perform Analysis with openAI and nltk

In [15]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [17]:
# Perform sentiment analysis
sid = SentimentIntensityAnalyzer()
sentiment_scores = sid.polarity_scores(conversation)

print(f"Sentiment Analysis:\nPositive: {sentiment_scores['pos']} | Negative: {sentiment_scores['neg']} | Neutral: {sentiment_scores['neu']}\n\n")

Sentiment Analysis:
Positive: 0.183 | Negative: 0.037 | Neutral: 0.78




In [19]:
import textstat

# Calculate the Flesch-Kincaid readability score
readability_score = textstat.flesch_reading_ease(conversation)

print(f"Readability Score (Flesch-Kincaid): {readability_score}\n\n")

Readability Score (Flesch-Kincaid): 88.33




In [13]:
import openai

openai.api_key = "<Your openAI Key goes here>"

messages=[
    {
      "role": "system",
      "content": """
        You will be provided with a conversation. Your task is to give a summary and mention all the main details in bullet points.
        Replace speaker 1 and speaker 2 with sales excutive or comapny name and customer name if available.
        """
    },
    {
      "role": "user",
      "content": conversation
    }
  ]


response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=0,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

- Customer wants to order flowers from Martha's Flowers Towne SST
- Customer's name is Randall Thomas
- Customer's phone number is 409-866-5088
- Customer's email address is Randall.Thomas@gmail.com
- Customer's shipping address is 6800 Gladys Avenue, Beaumont Texas, zip code 77706
- Customer wants to purchase one dozen long stem red roses
- Total amount of the order is $40
- Customer wants the roses to be delivered within 24 hours
- Customer doesn't need any further assistance
- Customer thanks Martha's Flowers and ends the call
